In [1]:
import findspark
findspark.init('/home/anycaroliny/spark-3.3.2-bin-hadoop3')

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('lr_ecample').getOrCreate()

23/03/24 15:11:07 WARN Utils: Your hostname, anycaroliny-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
23/03/24 15:11:07 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/24 15:11:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
from pyspark.ml.regression import LinearRegression

In [7]:
data = spark.read.csv('Ecommerce_Customers.csv', inferSchema=True, header=True)

In [8]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [19]:
# data.show()
# data.head(1)

for item in data.head(3)[2]:
    print(item)

pallen@yahoo.com
24645 Valerie Unions Suite 582Cobbborough, DC 99414-7564
Bisque
33.000914755642675
11.330278057777512
37.110597442120856
4.104543202376424
487.54750486747207


In [24]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [25]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [27]:
assembler = VectorAssembler(inputCols=[ 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership'],outputCol='features') # this is what to expect

In [28]:
output = assembler.transform(data)# transforming the data

In [30]:
type(output)
output.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)



In [41]:
features_list = [output.head(1)[0][4],output.head(1)[0][5],output.head(1)[0][6],output.head(1)[0][7]]
print(features_list)
# output.head(1)[0][8]

[12.65565114916675, 39.57766801952616, 4.0826206329529615, 587.9510539684005]


In [42]:
final_data = output.select('features', 'Yearly Amount Spent')

In [43]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [44]:
training_data, test_data = final_data.randomSplit([0.7,0.3])

In [45]:
training_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                341|
|   mean|  500.0632568497705|
| stddev|  79.61531464078932|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [46]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                159|
|   mean| 497.70722354417364|
| stddev|  78.89269644280506|
|    min|  275.9184206503857|
|    max|  725.5848140556806|
+-------+-------------------+



In [48]:
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [50]:
lr_model = lr.fit(training_data)

23/03/24 15:41:10 WARN Instrumentation: [fdab5f18] regParam is zero, which might cause numerical instability and overfitting.
23/03/24 15:41:10 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/03/24 15:41:10 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
23/03/24 15:41:10 WARN InstanceBuilder$NativeLAPACK: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [53]:
test_results = lr_model.evaluate(test_data)

In [54]:
test_results.residuals.show()

+--------------------+
|           residuals|
+--------------------+
|  11.024319385117451|
|-0.31891099351548746|
|  -5.037718024789285|
|  -6.028952903411778|
| -1.9734294086475757|
| -1.7836382491427116|
|  -3.927606037105761|
| -16.671766043973605|
|  -2.436565872269739|
|  11.474552612601713|
|   7.678927313969439|
| -17.637855542251373|
|   4.799398761974317|
|-0.23942238369733104|
|  -8.663540666469316|
|    4.20852003595644|
|   1.908954570929609|
|   5.461345954890817|
|   5.148969778975015|
|  16.801136869270408|
+--------------------+
only showing top 20 rows



In [55]:
test_results.rootMeanSquaredError

9.72076191285621

In [57]:
test_results.r2

0.9847219808498594

In [58]:
unlabeled_data = test_data.select('features')

In [59]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[31.1695067987115...|
|[31.2606468698795...|
|[31.5171218025062...|
|[31.5257524169682...|
|[31.5761319713222...|
|[31.7216523605090...|
|[31.8124825597242...|
|[31.8164283341993...|
|[31.8530748017465...|
|[31.9262720263601...|
|[31.9549038566348...|
|[31.9563005605233...|
|[31.9764800614612...|
|[32.0047530203648...|
|[32.0085045178551...|
|[32.0215955013870...|
|[32.0444861274404...|
|[32.0478009788678...|
|[32.0542618511847...|
|[32.0609143984100...|
+--------------------+
only showing top 20 rows



In [60]:
predictions = lr_model.transform(unlabeled_data)

In [61]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[31.1695067987115...|416.33221141717536|
|[31.2606468698795...|421.64554225046686|
|[31.5171218025062...|  280.956138675175|
|[31.5257524169682...|449.99457971329366|
|[31.5761319713222...| 543.2000133979759|
|[31.7216523605090...|349.56056488101535|
|[31.8124825597242...|  396.737951020903|
|[31.8164283341993...|   517.79425754763|
|[31.8530748017465...| 461.7216893346217|
|[31.9262720263601...| 380.7303808317247|
|[31.9549038566348...|432.31895262595754|
|[31.9563005605233...| 564.7637872894502|
|[31.9764800614612...| 325.7950472721259|
|[32.0047530203648...|463.98540350432677|
|[32.0085045178551...| 451.8607616952247|
|[32.0215955013870...|  517.363654721871|
|[32.0444861274404...|446.32087461562037|
|[32.0478009788678...|507.98922523120564|
|[32.0542618511847...|  556.725687890008|
|[32.0609143984100...| 610.8021818437446|
+--------------------+------------